# 一、大语言模型
大语言模型（LLM）是通过预测下一个词的监督学习方式进行训练的。具体来说，首先准备一个包含数百亿甚至更多词的大规模文本数据集。然后，可以从这些文本中提取句子或句子片段作为模型输入。模型会根据当前输入 Context 预测下一个词的概率分布。通过不断比较模型预测和实际的下一个词，并更新模型参数最小化两者差异,语言模型逐步掌握了语言的规律，学会了预测下一个词。

在训练过程中,研究人员会准备大量句子或句子片段作为训练样本,要求模型一次次预测下一个词，通过反复训练促使模型参数收敛，使其预测能力不断提高。经过在海量文本数据集上的训练，语言模型可以达到十分准确地预测下一个词的效果。这种以预测下一个词为训练目标的方法使得语言模型获得强大的语言生成能力。

大型语言模型主要可以分为两类:基础语言模型和指令调优语言模型。

基础语言模型（Base LLM）通过反复预测下一个词来训练的方式进行训练，没有明确的目标导向。因此，如果给它一个开放式的 prompt ，它可能会通过自由联想生成戏剧化的内容。而对于具体的问题，基础语言模型也可能给出与问题无关的回答。例如，给它一个 Prompt ，比如”中国的首都是哪里？“，很可能它数据中有一段互联网上关于中国的测验问题列表。这时，它可能会用“中国最大的城市是什么？中国的人口是多少？”等等来回答这个问题。但实际上，您只是想知道中国的首都是什么，而不是列举所有这些问题。

相比之下，指令微调的语言模型（Instruction Tuned LLM）则进行了专门的训练，以便更好地理解问题并给出符合指令的回答。例如，对“中国的首都是哪里？”这个问题，经过微调的语言模型很可能直接回答“中国的首都是北京”，而不是生硬地列出一系列相关问题。指令微调使语言模型更加适合任务导向的对话应用。它可以生成遵循指令的语义准确的回复，而非自由联想。因此，许多实际应用已经采用指令调优语言模型。熟练掌握指令微调的工作机制，是开发者实现语言模型应用的重要一步。

In [1]:
import openai
openai.api_key = "sk-75d7f4d4d4f744fb925f0bf09c87c48b"
openai.api_base = "https://api.deepseek.com"
# 下文第一个函数即tool工具包中的同名函数，此处展示出来以便于读者对比
def get_completion(prompt, model="deepseek-chat"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 控制模型输出的随机程度
    )
    return response.choices[0].message["content"]

In [2]:
response = get_completion("中国的首都是哪里？")
print(response)

中国的首都是**北京**。  
北京是中国的政治、文化、国际交往和科技创新中心，也是中华人民共和国中央人民政府所在地。作为历史悠久的名城，北京拥有故宫、天坛、长城等世界文化遗产，同时也是一个现代化国际大都市。


这也就是训练一个指令微调语言模型（例如ChatGPT）的过程。 首先，在大规模文本数据集上进行无监督预训练，获得基础语言模型。 这一步需要使用数千亿词甚至更多的数据，在大型超级计算系统上可能需要数月时间。 之后，使用包含指令及对应回复示例的小数据集对基础模型进行有监督 fine-tune，这让模型逐步学会遵循指令生成输出，可以通过雇佣承包商构造适合的训练示例。 接下来，为了提高语言模型输出的质量，常见的方法是让人类对许多不同输出进行评级，例如是否有用、是否真实、是否无害等。 然后，您可以进一步调整语言模型，增加生成高评级输出的概率。这通常使用基于人类反馈的强化学习（RLHF）技术来实现。 相较于训练基础语言模型可能需要数月的时间，从基础语言模型到指令微调语言模型的转变过程可能只需要数天时间，使用较小规模的数据集和计算资源。

# 二、tokens
到目前为止对 LLM 的描述中，我们将其描述为一次预测一个单词，但实际上还有一个更重要的技术细节。即 LLM 实际上并不是重复预测下一个单词，而是重复预测下一个 token 。对于一个句子，语言模型会先使用分词器将其拆分为一个个 token ，而不是原始的单词。对于生僻词，可能会拆分为多个 token 。这样可以大幅降低字典规模，提高模型训练和推断的效率。例如，对于 "Learning new things is fun!" 这句话，每个单词都被转换为一个 token ，而对于较少使用的单词，如 "Prompting as powerful developer tool"，单词 "prompting" 会被拆分为三个 token，即"prom"、"pt"和"ing"。

In [3]:
# 为了更好展示效果，这里就没有翻译成中文的 Prompt
# 注意这里的字母翻转出现了错误，吴恩达老师正是通过这个例子来解释 token 的计算方式
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)


Let's take the word "lollipop" and reverse its letters.

Original: l o l l i p o p

Reversed: p o p i l l o l

So, the reversed word is "popillol".

However, note that "lollipop" is often spelled with two 'l's at the beginning and two 'p's at the end, but when reversed, it should be straightforward.

Let me write it clearly:
- Forward: l-o-l-l-i-p-o-p
- Backward: p-o-p-i-l-l-o-l

Thus, the reversed string is "popillol". 

If you meant to reverse the sequence of letters without considering the word structure, this is correct. If you intended something else, let me know!


分词方式也会对语言模型的理解能力产生影响。当您要求 ChatGPT 颠倒 "lollipop" 的字母时，由于分词器（tokenizer） 将 "lollipop" 分解为三个 token，即 "l"、"oll"、"ipop"，因此 ChatGPT 难以正确输出字母的顺序。这时可以通过在字母间添加分隔，让每个字母成为一个token，以帮助模型准确理解词中的字母顺序。

In [4]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

print(response)


Let's take the word "l-o-l-l-i-p-o-p" and reverse the order of its letters.

Original: l - o - l - l - i - p - o - p

Now, reversing the sequence:
Start from the end: p, then o, then p, then i, then l, then l, then o, then l.

So, the reversed order is: p - o - p - i - l - l - o - l

Therefore, the reversed string is "popillol".

To write it without hyphens: popillol


因此,语言模型以 token 而非原词为单位进行建模，这一关键细节对分词器的选择及处理会产生重大影响。开发者需要注意分词方式对语言理解的影响，以发挥语言模型最大潜力。



❗❗❗ 对于英文输入，一个 token 一般对应 4 个字符或者四分之三个单词；对于中文输入，一个 token 一般对应一个或半个词。不同模型有不同的 token 限制，需要注意的是，这里的 token 限制是输入的 Prompt 和输出的 completion 的 token 数之和，因此输入的 Prompt 越长，能输出的 completion 的上限就越低。 ChatGPT3.5-turbo 的 token 上限是 4096。

